In [1]:
import numpy as np
from xgboost import XGBClassifier
from keras.utils import to_categorical
import pandas as pd

### functions

In [2]:
def onehot(y):
    label = []
    for i in y:
        label.append([int(i)])
    y_result = to_categorical(label).astype(int)
    
    return y_result

In [3]:
def load_csv(csv_path):
    dataframe = pd.read_csv(csv_path)
    df_to_process = dataframe.copy()
    
    # drop frame name
    df_to_process.drop(columns=['file_name'], inplace=True)
    
    # pop class names
    df_to_process.pop('class_name')
    
    # extract the labels
    y = df_to_process.pop('class_no')

    # convert the input features and labels into the correct format for training
    X = df_to_process.astype('float64')
    y = to_categorical(y)
    return X.values, y
    

### extract keypoints using confidence score

In [4]:

def extract_kpts(X, conf_threshold):
    kpts =[]
    for kpts_array in X:
        num_kpts = len(kpts_array) // 3
        temp = []
        for kid in range(num_kpts - 6):
            x_coords, y_coords = kpts_array[3 * kid], kpts_array[3 * kid + 1]
            conf = kpts_array[3 * kid + 2]
            if conf < conf_threshold:
                x_coords, y_coords = 0.0, 0.0
                temp.extend([x_coords, y_coords])
                continue
            temp.extend([x_coords, y_coords])
        kpts.append(temp)
        
    return kpts

In [6]:
X_train, y_train = load_csv('../data/csvs/origin/train_data_mv5.csv')
X_test, y_test = load_csv('../data/csvs/origin/test_data_mv5.csv')
kpts_train = extract_kpts(X_train, 0.3)
kpts_test = extract_kpts(X_test, 0.3)

In [7]:
# change array to dataframe
kpts_train_df = pd.DataFrame(kpts_train)
y_train_df = pd.DataFrame(y_train)
kpts_test_df = pd.DataFrame(kpts_test)
y_test_df = pd.DataFrame(y_test)

In [8]:
kpts_train_df.to_csv('train_x_mv5.csv', index=False)
y_train_df.to_csv('train_y_mv5.csv', index=False)
kpts_test_df.to_csv('test_x_mv5.csv', index=False)
y_test_df.to_csv('test_y_mv5.csv', index=False)